In [290]:
import os.path
import random
from matplotlib import pyplot, image
from pathlib import Path
import numpy as np
from PIL import Image
import dask.dataframe as dd
from dask.distributed import Client
from dask.distributed import get_worker
from dask.distributed import progress
from fabric import Connection, Config

In [291]:
client = Client("tcp://128.110.217.91:8786")

/users/prsridha/.local/lib/python3.8/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.7.0  | 1.7.0     | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [292]:
client

<Client: 'tcp://128.110.217.91:8786' processes=2 threads=32, memory=125.38 GiB>

In [293]:
DATA_URL = "metadata.csv"
feature_names = ["image_path", "label"]
dtypes = {'image_path': np.str, 'label': np.int16}
df0 = dd.read_csv(DATA_URL, names=feature_names, dtype=dtypes)
df = df0.sample(frac=0.001)

/tmp/ipykernel_352036/3867680041.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtypes = {'image_path': np.str, 'label': np.int16}


In [294]:
len(df)

42

In [295]:
def initialize_workers(client):
    """Initialize workers (get worker IPs)"""
    worker_id_ip_dict = {}
    all_worker_details = client.scheduler_info()['workers']
    i = 0
    for ip in all_worker_details:
        # set the mapping between worker ID and worker IP
        worker_id_ip_dict[i] = str(ip)
        i += 1
    return worker_id_ip_dict
workers = initialize_workers(client)

In [296]:
def create_connection():    
    user = "prsridha"
    host = "128.110.217.91"
    pem_path = "./cloudlab.pem"
    connect_kwargs = {"key_filename":pem_path}
    conn = Connection(host, user=user, connect_kwargs=connect_kwargs)
    
    return conn

In [297]:
def pull(from_path, to_path, conns):
    worker_id = get_worker().address
    conn = conns[worker_id]
    to_path_dir = "/".join(to_path.split("/")[:-1])
    Path(to_path_dir).mkdir(parents=True, exist_ok=True)
    result = conn.get(from_path, to_path)
    print("Pulled from {0.remote} to {0.local}".format(result))

In [298]:
shuffled_df = df.sample(frac=1)
shuffled_df.head()

,image_path,label
15022,data/trainingSet/trainingSet/3/img_4185.jpg,3
11433,data/trainingSet/trainingSet/2/img_22909.jpg,2
15025,data/trainingSet/trainingSet/3/img_35701.jpg,3
20074,data/trainingSet/trainingSet/4/img_27296.jpg,4
3467,data/trainingSet/trainingSet/0/img_8857.jpg,0


In [299]:
new_df = shuffled_df.repartition(npartitions=2)

In [300]:
def preprocess_row(row):
    return str(row["image_path"]) + ";" + str(row["label"])

In [301]:
def preprocess_col(path, conns):
    new_path = "/users/prsridha/" + path
    pull(new_path, new_path, conns)
    
    im = Image.open(str(new_path))
    pixels = list(im.getdata())
    return sum(pixels)

In [307]:
conns = {}
for i in workers.values():
    fut_conn = client.submit(create_connection, workers=i)
    conns[i] = fut_conn.result()

In [303]:
def preprocess_col1(path, conns):
    worker_id = get_worker().address
    print("Running path",path, "on worker", worker_id)
    if not os.path.isfile(path):
        to_path_dir = "/".join(path.split("/")[:-1])
        Path(to_path_dir).mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        f.write("Created file " + path)
    return 0

In [304]:
def preprocess_col2(path, conns):
    worker_id = get_worker().address
    print("Running path",path, "on worker", worker_id)
    return worker_id

In [305]:
new_col = new_df.image_path.map_partitions(lambda x: x.apply(lambda y: preprocess_col2(str(y), conns)), meta=('new_col', int))

In [306]:
new_col.compute(workers=[workers[1], workers[0]])

15022    tcp://128.110.217.73:44061
11433    tcp://128.110.217.73:44061
15025    tcp://128.110.217.73:44061
20074    tcp://128.110.217.73:44061
3467     tcp://128.110.217.73:44061
41313    tcp://128.110.217.73:44061
37900    tcp://128.110.217.73:44061
4404     tcp://128.110.217.73:44061
5297     tcp://128.110.217.73:44061
40472    tcp://128.110.217.73:44061
731      tcp://128.110.217.73:44061
39643    tcp://128.110.217.73:44061
15361    tcp://128.110.217.73:44061
19988    tcp://128.110.217.73:44061
6431     tcp://128.110.217.73:44061
39707    tcp://128.110.217.73:44061
33296    tcp://128.110.217.73:44061
16268    tcp://128.110.217.73:44061
12421    tcp://128.110.217.73:44061
16819    tcp://128.110.217.73:44061
34055    tcp://128.110.217.73:44061
9329     tcp://128.110.217.73:44061
18824    tcp://128.110.217.73:44061
40183    tcp://128.110.217.73:44061
27681    tcp://128.110.217.73:44061
29958    tcp://128.110.217.73:44061
14516    tcp://128.110.217.73:44061
36974    tcp://128.110.217.7

In [45]:
part1 = new_df.partitions[0]
part2 = new_df.partitions[1]

In [46]:
n1 = list(part1.image_path.compute())
n2 = list(part2.image_path.compute())
with open("worker1_list.txt", "w") as f:
    f.write("\n".join(n1))
with open("worker2_list.txt", "w") as f:
    f.write("\n".join(n2))

In [26]:
new_col1 = part1.image_path.map_partitions(lambda x: x.apply(lambda y: preprocess_col(str(y))), meta=('new_col1', int))
new_col2 = part2.image_path.map_partitions(lambda x: x.apply(lambda y: preprocess_col(str(y))), meta=('new_col2', int))

In [93]:
new_col1.compute(workers=workers[0])

In [94]:
new_col2.compute(workers=workers[1])

In [ ]:
conn.close()
